Hyperparameters Tuning

In [2]:
from xgboost import XGBRegressor

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from xgb_params import xgb_params

# GridSearchCV

In [3]:
params = xgb_params
# Import train data
process_train_path = './processed/processed_train_2.csv'
df = pd.read_csv(process_train_path)
y_train = df['price_doc']
X_train = df.drop(columns=['price_doc'])

In [5]:
from sklearn.model_selection import GridSearchCV

regressor =  XGBRegressor(
                    device=params['device'],
                    objective=params['objective'],
                    eval_metric=params['eval_metric'],
                    enable_categorical=params['enable_categorical'],
                    # early_stopping_rounds=params['early_stopping_rounds'],
                    eta=params['eta'],
                    n_jobs=params['n_jobs'],

                    n_estimators=params['n_estimators'],
                    max_depth = params['max_depth'],
                    colsample_bytree=params['colsample_bytree'],
                )

param_grid = {
            #   "max_depth": [5,6,8],
            #   "n_estimators": [100, 200, 300],
            #   "colsample_bytree": [0.7, 0.8]
              # "learning_rate": [0.05, 0.1, 0.2],
              "gamma": [0, 8],
              "reg_alpha": [0, 93],
              "min_child_weight": [0, 2],
              }

search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

print(f"The best hyperparams are {search.best_params_}")

c:\Users\Andre\Desktop\Git\CZ4041-Sterbank-Market\venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\Users\Andre\Desktop\Git\CZ4041-Sterbank-Market\venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\Users\Andre\Desktop\Git\CZ4041-Sterbank-Market\venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\Users\Andre\Desktop\Git\CZ4041-Sterbank-Market\venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version.

The best hyperparams are {'colsample_bytree': 0.7, 'max_depth': 6, 'n_estimators': 300}


# HyperOpt

In [ ]:
# Import train data
process_train_path = './processed/processed_train_2.csv'
df = pd.read_csv(process_train_path)
# Split data into train and test
# X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['price_doc']), df['price_doc'], test_size=0.25, random_state=42)
# train_df = pd.concat([X_train, y_train], axis=1)
# processed_df, oe, te = process_train(train_df)
# processed_df.shape



is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


(11813, 286)

In [5]:
space={
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'eta': hp.uniform('eta', 0.01, 0.2),
    }

In [6]:
def objective(space):
    model = XGBRegressor(
                    n_estimators=params2['n_estimators'],
                    device=params2['device'],
                    objective=params2['objective'],
                    eval_metric=params2['eval_metric'],
                    enable_categorical=params2['enable_categorical'],
                    early_stopping_rounds=params2['early_stopping_rounds'],

                    eta=space['eta'],
                    max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree'])
                )
    evaluation = [( X_train, y_train), ( X_test, y_test)]

    model.fit(X_train, y_train, eval_set=evaluation)

    pred = model.predict(X_test)
    rmsle = mean_squared_log_error(y_test, pred, squared=False)
    r2 = r2_score(y_test, pred)

    print(f"RMSE: {rmsle}, R2: {r2}")
    return {'loss': rmsle, 'status': STATUS_OK}

In [7]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is 

[0]	validation_0-rmsle:0.63117	validation_1-rmsle:0.64835
[1]	validation_0-rmsle:0.62709	validation_1-rmsle:0.64450
[2]	validation_0-rmsle:0.62709	validation_1-rmsle:0.64448
[3]	validation_0-rmsle:0.62475	validation_1-rmsle:0.64251
[4]	validation_0-rmsle:0.62201	validation_1-rmsle:0.63990
[5]	validation_0-rmsle:0.62120	validation_1-rmsle:0.63907
[6]	validation_0-rmsle:0.62070	validation_1-rmsle:0.63867
[7]	validation_0-rmsle:0.61808	validation_1-rmsle:0.63607
[8]	validation_0-rmsle:0.61799	validation_1-rmsle:0.63603
[9]	validation_0-rmsle:0.61444	validation_1-rmsle:0.63294
[10]	validation_0-rmsle:0.61224	validation_1-rmsle:0.63086
[11]	validation_0-rmsle:0.60929	validation_1-rmsle:0.62791
[12]	validation_0-rmsle:0.60864	validation_1-rmsle:0.62731
[13]	validation_0-rmsle:0.60857	validation_1-rmsle:0.62728
[14]	validation_0-rmsle:0.60188	validation_1-rmsle:0.61991
[15]	validation_0-rmsle:0.60122	validation_1-rmsle:0.61928
[16]	validation_0-rmsle:0.59502	validation_1-rmsle:0.61502
[17]	va

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

[23:10:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predic

[0]	validation_0-rmsle:0.63014	validation_1-rmsle:0.64735                        
[1]	validation_0-rmsle:0.62412	validation_1-rmsle:0.64161                        
[2]	validation_0-rmsle:0.62412	validation_1-rmsle:0.64157                        
  1%|          | 1/100 [00:03<04:29,  2.72s/trial, best loss: 0.5754723707594527]

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.



[3]	validation_0-rmsle:0.62115	validation_1-rmsle:0.63900                        
[4]	validation_0-rmsle:0.61737	validation_1-rmsle:0.63530                        
[5]	validation_0-rmsle:0.61626	validation_1-rmsle:0.63418                        
[6]	validation_0-rmsle:0.61559	validation_1-rmsle:0.63363                        
[7]	validation_0-rmsle:0.61212	validation_1-rmsle:0.63013                        
[8]	validation_0-rmsle:0.61199	validation_1-rmsle:0.63006                        
[9]	validation_0-rmsle:0.60744	validation_1-rmsle:0.62610                        
[10]	validation_0-rmsle:0.60482	validation_1-rmsle:0.62364                       
[11]	validation_0-rmsle:0.60133	validation_1-rmsle:0.62004                       
[12]	validation_0-rmsle:0.60066	validation_1-rmsle:0.61933                       
[13]	validation_0-rmsle:0.60056	validation_1-rmsle:0.61928                       
[14]	validation_0-rmsle:0.59040	validation_1-rmsle:0.60796                       
[15]	validation_

KeyboardInterrupt: 

In [ ]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.7735799874380092, 'eat': 0.1996895442801695, 'gamma': 8.084029345968737, 'max_depth': 17.0, 'min_child_weight': 2.0, 'reg_alpha': 93.0, 'reg_lambda': 0.8685796539747039}
